In [10]:
#classify order or categorize data into one or more of set of classes

In [11]:
# fake news classify trained to detect is if fake or not.

In [12]:
# import libraries

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
!pip install -q kaggle

In [22]:
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json 

Saving kaggle.json to kaggle.json
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [28]:
!pip install --upgrade --force-reinstall --no-deps kaggle
!pip install --upgrade --force-reinstall --no-deps kaggle-cli

  Using cached kaggle-1.5.12-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
  Created wheel for kaggle-cli: filename=kaggle_cli-0.12.13-py3-none-any.whl size=9487 sha256=38a01f2cba14964db3475685ded1c360b665148563413b377169a0ba7a1e2d44
  Stored in directory: /root/.cache/pip/wheels/76/b7/49/f439aedefd1839a586a54a03f30d5b9a5320c703cbee1631be
Successfully built kaggle-cli


In [31]:
!kaggle competitions download -c fake-news

 88% 41.0M/46.5M [00:00<00:00, 49.2MB/s]
100% 46.5M/46.5M [00:00<00:00, 51.6MB/s]


In [33]:
!unzip fake-news.zip


Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [34]:
import pandas as pd
data = pd.read_csv("train.csv")
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [35]:
# remove NA
data = data.dropna()
data.reset_index(inplace=True)

In [36]:
X = data.drop("label",axis=1)
Y = data["label"]

In [37]:
# checking the shape of X and Y
print(X.shape)
print(Y.shape)

(18285, 5)
(18285,)


In [39]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential 

# preprocessing the text
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [40]:
messages = X.copy()

In [41]:
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
  review = re.sub("[^a-zA-Z]"," ",messages["title"][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word)  for word in review if not word in stopwords.words("english")] # (word not in stopwords) and stem(word)
  review = " ".join(review)
  corpus.append(review)

In [42]:
# one hot encoding

In [43]:
vocab_size = 5000
onehot_repr =[one_hot(words,vocab_size) for words in corpus]

In [44]:
# apply pad sequences to make sentence length equal
sent_length =20
embedded_docs = pad_sequences(onehot_repr,padding="pre",maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4805 2240 1998]
 [   0    0    0 ... 4372 2551 3826]
 [   0    0    0 ...  229 3077 2848]
 ...
 [   0    0    0 ... 1638 1871 3174]
 [   0    0    0 ...   55  559 4784]
 [   0    0    0 ... 3519   25 3223]]


In [45]:
# creating model for train set

In [46]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [48]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(Y)


In [49]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [50]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 35ms/step - loss: 0.3380 - accuracy: 0.8362 - val_loss: 0.1999 - val_accuracy: 0.9155
Epoch 2/10
192/192 [==============================] - 6s 32ms/step - loss: 0.1421 - accuracy: 0.9458 - val_loss: 0.2036 - val_accuracy: 0.9123
Epoch 3/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0952 - accuracy: 0.9649 - val_loss: 0.2382 - val_accuracy: 0.9112
Epoch 4/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0601 - accuracy: 0.9793 - val_loss: 0.2591 - val_accuracy: 0.9208
Epoch 5/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0387 - accuracy: 0.9874 - val_loss: 0.3142 - val_accuracy: 0.9186
Epoch 6/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0231 - accuracy: 0.9929 - val_loss: 0.3855 - val_accuracy: 0.9162
Epoch 7/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0110 - accuracy: 0.9975 - val_loss: 0.4799 - val_accuracy: 0.9186

In [52]:
# performance metrics and accuracy
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3119,  300],
       [ 227, 2389]])

In [53]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9126760563380282

In [54]:
# tareas


Tareas

Try different test processing techniques
train fake classify from more data
Try different neural architechture
